In [1]:
!pip install tabula-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 16.2 MB/s eta 0:00:00


In [ ]:
import tabula
import json
# Path to your PDF file
pdf_path = '/content/42xxx_xxxxxxx_xxxx.pdf'

In [ ]:
# Use tabula to read tables from the PDF
# This returns a list of DataFrame objects, one for each table detected in the PDF.
tables = tabula.read_pdf(pdf_path, pages="all", multiple_tables=True)
# Convert the table data to JSON
# This will create a list of JSON objects if there are multiple tables
json_tables = [table.to_json(orient="records") for table in tables]
# Load the JSON data of the first table into Python dictionary objects
table_data_1=json.loads(json_tables[0])
# Convert the second row of the first table to a list of (column_name, value) tuples
u=list(table_data_1[1].items())
print(u)
# Convert the fourth row of the first table to a list of (column_name, value) tuples
v=list(table_data_1[3].items())
print(v)
# Convert the third row of the first table to a list of (column_name, value) tuples
t=list(table_data_1[2].items())
print(t)

In [ ]:
# Load the JSON data of the second table into Python dictionary objects
table_data_2 = json.loads(json_tables[1])
# Load the JSON data of the third table into Python dictionary objects
table_data_3 = json.loads(json_tables[2])
# Convert the first row of the second table to a list of (column_name, value) tuples
e=list(table_data_2[0].items())
print(e)
# Convert the second row of the second table to a list of (column_name, value) tuples
d=list(table_data_2[1].items())
print(d)
# Convert the first row of the third table to a list of (column_name, value) tuples
z = list(table_data_3[0].values())
print(z)

In [ ]:
if len(e)==1 :
   # If the length of the list 'e' is 1, extract the relevant information from the first item of each list.
  desc = e[0][1].split('Platform/MODU/project name ')[1]
  mc = d[0][1].split('OCS-G block and area ')[1]
  cname = u[0][1].split('Name of operator contact ')[1]
  cemail = v[0][1].split('E-mail address for notifications ')[1]
  cphone = t[0][1].split('Phone number of operator contact ')[1]
  stattype = e[0][0].split('Station type (Spar, TLP, Semi, MODU, Mooring) ')[1]
  ins = z[0].split('Instrument model (e.g. RDI 75kHz BB) ')[1]
elif len(e)==2:
   # If the length of the list 'e' is 2, check if the second item's value is None.
  if e[1][1] is None:
    # If the second item's value is None, extract the relevant information from the first item of each list.
    desc = e[0][1].split('Platform/MODU/project name ')[1]
    mc = d[0][1].split('OCS-G block and area ')[1]
    cname = u[0][1].split('Name of operator contact ')[1]
    cemail = v[0][1].split('E-mail address for notifications ')[1]
    cphone = t[0][1].split('Phone number of operator contact ')[1]
    stattype = e[0][0].split('Station type (Spar, TLP, Semi, MODU, Mooring) ')[1]
    ins = z[0].split('Instrument model (e.g. RDI 75kHz BB) ')[1]
  else:
    # If the second item's value is not None, extract the relevant information directly from the second item of each list.
    desc = e[1][1]
    mc = d[1][1]
    cname = u[1][1]
    cemail = v[1][1]
    cphone = t[1][1]
    stattype = e[1][0]
    ins = z[1]
# Now you have extracted the following information:
# desc: Description of the platform/MODU/project name
# mc: OCS-G block and area
# cname: Name of the operator contact
# cemail: E-mail address for notifications
# cphone: Phone number of the operator contact
# stattype: Station type (Spar, TLP, Semi, MODU, Mooring)
# ins: Instrument model (e.g. RDI 75kHz BB)

In [ ]:
# Extract the fourth row of the second table as a list of values
x = list(table_data_2[3].values())
# Check if the first item of the list 'x' is 'Latitude (Deg, Min, Sec)'
if x[0] == 'Latitude (Deg, Min, Sec)' :
  # If true, remove ' N' from the second item and strip any leading/trailing whitespace
  lat = list(x)[1].replace(' N','').strip()
else :
  # Otherwise, split the first item at 'Latitude (Deg, Min, Sec) ' and remove trailing spaces
  lat = x[0].split('Latitude (Deg, Min, Sec) ')[1].rsplit(' ', 1)[0]
# Extract the fifth row of the second table as a list of values
y = list(table_data_2[4].values())
# Check the length of the list 'y' and handle accordingly
if len(y) == 1:
  # If there's only one item, split and extract the longitude value
  lon = y[0].split('Longitude (Deg, Min, Sec) ')[1]
elif len(y) == 2 and y[1] is None:
  # If there are two items and the second item is None, extract from the first item
  lon = y[0].split('Longitude (Deg, Min, Sec) ')[1]
elif len(y) >= 2:
  # If there are two or more items, remove ' W' from the second item and strip whitespace
  lon = y[1].replace(' W', '').strip()
# Define a function to convert DMS (Degrees, Minutes, Seconds) to decimal degrees
def dms_to_decimal(dms):
    # Remove common non-numeric characters
    dms = dms.replace('°', ' ').replace('’', ' ').replace('\'', ' ').replace('”', ' ').replace('"', ' ')
    parts = dms.split()
    #print(parts)
    degrees = float(parts[0])
    minutes = float(parts[1])
    seconds = float(parts[2])
    # Convert DMS to decimal degrees
    decimal = degrees + (minutes / 60) + (seconds / 3600)
    return decimal
# Example usage
lat_dms = lat
lon_dms = lon
# Convert the DMS values to decimal degrees
lat_decimal = dms_to_decimal(lat_dms)
lon_decimal = dms_to_decimal(lon_dms)
# Round the decimal values to 4 decimal places
lat_val = round(lat_decimal, 4)
lon_val = round(lon_decimal, 4)

In [ ]:
# Initialize an empty list to hold all transformed data from each table
all_transformed_data = []
#Iterate over each JSON string in json_tables
for json_table in json_tables:
     # Parse the JSON string back into a Python object (list of dictionaries)
     table_data = json.loads(json_table)
     # Initialize an empty dictionary for the current table's transformed data
     transformed_data = {}
     # Iterate over each row in the current table
     for row in table_data:
         # Check if the row has at least two values
         if len(row.values()) >= 2:
           # Extract the first key and value from the row
             key = list(row.keys())[0]  # The descriptor (e.g., "Operator URL")
             value = list(row.values())[0]  # The corresponding detail (e.g., "www.shell.com")
             # Add the key-value pair to the transformed_data dictionary
             transformed_data[key] = value
     # Add the transformed data for the current table to the list
     if transformed_data:
         all_transformed_data.append(transformed_data)
    # Now you have a list 'all_transformed_data' containing dictionaries with transformed data from each table

In [ ]:
# Extracting the PDF name from the path
pdf_name = pdf_path.split('/')[-1]

# Getting the first five characters of the PDF name
platform_name = pdf_name[:5]
print(platform_name)
print(desc)
print(mc)
print(cname)
print(cemail)
print(cphone)
print(stattype)
print(ins)
print(lat_val)
print(lon_val)

UPDATE platform SET description = '42902 - Deepwater Proteus - MC809', local = '42902 - Deepwater Proteus - MC809', loc_lat = 28.159, loc_lon = -89.1322, contactName = 'Keith Kurrus', contactEmail = 'HoustonOpsEHI@rpsgroup.com', Phone = '281-495-0883', ttype = 'MODU' WHERE name = '42902'
UPDATE sensor SET instrument='RDI 38kHz DR OO III' WHERE rowid= and platformId= ;
